In [23]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [24]:
root_dir = '../slices'

In [26]:
slice_df = pd.read_csv("../slice_filenames.csv")
slice_df.head()

,Unnamed: 0,fold,filename,classID
0,0,1,46654-6-0-0.wav_0.npy,6
1,1,1,46654-6-0-0.wav_1.npy,6
2,2,1,46654-6-0-0.wav_0.npy,6
3,3,1,46654-6-0-0.wav_1.npy,6
4,4,1,57320-0-0-24.wav_0.npy,0


In [27]:
class SignalDataset(Dataset):
    def __init__(self, slice_df):
        self.slice_df = slice_df

    def __len__(self):
        return len(self.slice_df)

    def __getitem__(self, idx):
        row = self.slice_df[idx]
        filename = row['filename']
        fold = row['fold']
        x = np.load(os.path.join(root_dir, f"fold{fold}", filename))
        return x, row['classID']